### Import the CSVs being used or you can skip this and use whatever dataframe is loaded in and import the Weather CSV

In [1]:
import numpy as np
import pandas as pd
# read in data from csv file to pandas dataframe.  
df_46A = pd.read_csv('df_46A.csv',  keep_default_na=True, sep=',\s+', delimiter=',', skipinitialspace=True)
df_wd = pd.read_csv('weather_data_final.csv',  keep_default_na=True, sep=',\s+', delimiter=',', skipinitialspace=True)

### Quickly rename the headers

In [2]:
df_wd.rename({'hourly_data_time':'DateTime','hourly_data_precip_intensity':'Precip Intensity',
             'hourly_data_precip_probability':'Precip Prob', 'hourly_data_apparent_temperature':'Temperature',
             'hourly_data_wind_speed':'Wind Speed', 'hourly_data_cloud_cover':'Cloud Cover',
             'hourly_data_uv_index':'UV Index','hourly_data_visibility':'Visibility'},axis=1,inplace=True)


In [3]:
# Precip Intensity has a mean ~ 0 so drop as it will skew predicition
df_wd_feat = df_wd.drop(columns=['Precip Intensity'])

### Now set both columns to datetime for merging later

In [4]:
df_wd_feat['DateTime']= pd.to_datetime(df_wd_feat['DateTime'], infer_datetime_format=True) 
df_46A['DateTime']= pd.to_datetime(df_46A['DateTime'], infer_datetime_format=True) 

### The mask is used to find all the weather data in December so use this cell to find whatever date range you want

In [5]:
mask = (df_wd_feat['DateTime'] > '2018-12-01') & (df_wd_feat['DateTime'] <= '2018-12-31')

In [6]:
df_wd_dec = df_wd_feat.loc[mask]

In [7]:
df_46A.iloc[0]

DateTime          2018-12-24 00:00:00
Sequence                           14
StopID                           6059
Planned_Arr                     54291
Actual_Arr                      54158
Actual_Depart                   54244
Extracted_Date              24-DEC-18
Day                            Monday
Day_Friday                          0
Day_Monday                          1
Day_Saturday                        0
Day_Sunday                          0
Day_Thursday                        0
Day_Tuesday                         0
Day_Wednesday                       0
Name: 0, dtype: object

In [8]:
df_wd_dec.iloc[551]

DateTime       2018-12-24 00:00:00
Precip Prob                      0
Temperature                  39.46
Wind Speed                    5.61
Cloud Cover                    0.6
UV Index                         0
Visibility                    6.22
Name: 17319, dtype: object

In [9]:
df_46A.iloc[0].DateTime == df_wd_dec.iloc[551].DateTime

True

### Pick the correct date to match and then merge the weather rows to the One Hot Encoded rows in the travel data

In [10]:
df_46A_merge = pd.merge(df_46A, df_wd_dec, how='outer', on='DateTime')

In [11]:
df_46A_merge.head(5)

,DateTime,Sequence,StopID,Planned_Arr,Actual_Arr,Actual_Depart,Extracted_Date,Day,Day_Friday,Day_Monday,...,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Precip Prob,Temperature,Wind Speed,Cloud Cover,UV Index,Visibility
0,2018-12-24,14.0,6059.0,54291.0,54158.0,54244.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
1,2018-12-24,15.0,334.0,54462.0,54348.0,54401.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
2,2018-12-24,16.0,406.0,54728.0,54588.0,54646.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
3,2018-12-24,17.0,747.0,54875.0,54773.0,54820.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
4,2018-12-24,18.0,842.0,54961.0,54964.0,54983.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22


In [12]:
# The weather data contains NaN values so replacing with the mean
df_46A_merge = df_46A_merge.fillna(df_46A_merge.mean())

In [13]:
df_46A_merge.head(5)

,DateTime,Sequence,StopID,Planned_Arr,Actual_Arr,Actual_Depart,Extracted_Date,Day,Day_Friday,Day_Monday,...,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,Precip Prob,Temperature,Wind Speed,Cloud Cover,UV Index,Visibility
0,2018-12-24,14.0,6059.0,54291.0,54158.0,54244.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
1,2018-12-24,15.0,334.0,54462.0,54348.0,54401.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
2,2018-12-24,16.0,406.0,54728.0,54588.0,54646.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
3,2018-12-24,17.0,747.0,54875.0,54773.0,54820.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22
4,2018-12-24,18.0,842.0,54961.0,54964.0,54983.0,24-DEC-18,Monday,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,39.46,5.61,0.6,0.0,6.22


## Check if they were merged correctly